In [9]:
pip install pymysql

  Using cached PyMySQL-1.1.0-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.0-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pymysql
from dotenv import load_dotenv
import os

In [2]:
dotenv_path = './config/keys.env'

In [9]:
#이 셀은 감정 추출하는 함수의 일부가 되어야 할 부분이라 따로 함수로 만들지 않음
user_id ='test' #이건 음성데이터와 별개로 SQL 문을 쓰기 위해 받아야 하는 값. 여기서는 임의로 이렇게 정의
emotions_prob_list = ['Angry_prob','Disgust_prob','Fear_prob','Happiness_prob','Neutral_prob','Sadness_prob','Surprise_prob']
cnt = 0

#이 for문은 실제 구현할 땐 변수 설정만 남겨놓고 집어넣는 값은 모델에서 나온 값으로
for i in emotions_prob_list:
    locals()[f'{i}'] = 0 + round(cnt,2) 
    cnt+=1/21 
    
emotions_list = [i[:-5] for i in emotions_prob_list]
emotions_dict={}
for i in range(len(emotions_list)):
    emotions_dict[locals()[f'{emotions_prob_list[i]}']] = emotions_list[i]   

#가장 값이 높은 감정의 이름을 도출하여
emotion  = emotions_dict[max(emotions_dict.keys())]

In [12]:
#DB에 보낸다.
def insert_emotion(emotion=emotion,user_id=user_id):
    load_dotenv(dotenv_path)
    host, port, user, password, db = os.getenv('MYSQL_HOST'),int(os.getenv('MYSQL_PORT')),os.getenv('MYSQL_USER'),os.getenv('MYSQL_PASSWORD'),os.getenv('MYSQL_DATABASE')
    conn = pymysql.connect(host=host,port=port, user=user, 
                           password=password, db=db, charset='utf8')
    try:
        with conn.cursor() as cursor:
            INSERT_SQL = """
                INSERT INTO TB_EMOTION (USER_ID, EMOTION_VAL) VALUES (%s, %s);
            """
            cursor.execute(INSERT_SQL, (user_id, emotion))
            conn.commit()
    finally:
        conn.close()

In [13]:
insert_emotion(emotion)